<a href="https://colab.research.google.com/github/Chandu1722/Hybrid-Encryption-Project-RSA-AES-/blob/main/RSA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random as rd
import math

# --- Helper Functions ---

def is_prime(x):
    """Checks if a number x is prime."""
    if x <= 1:
        return False
    # We only need to check for divisors up to the square root of x
    root = math.ceil(math.sqrt(x))
    for i in range(2, int(root) + 1):
        if x % i == 0:
            return False
    return True

def generate_prime():
    """Generates a prime number (based on the original notebook's logic)."""
    # Start with a random number between 2 and 100
    x = rd.randint(67, 314)
    # Keep incrementing until a prime number is found
    while not is_prime(x):
        x += 1
    return x

def gcd(a, b):
    """Calculates the greatest common divisor of a and b using the Euclidean algorithm."""
    while b != 0:
        a, b = b, a % b
    return a

def generate_public_key(totient):
    """
    Generates a public key 'e' such that 1 < e < totient
    and gcd(e, totient) == 1 (e is coprime to totient).
    """
    e = 2
    while e < totient:
        if gcd(e, totient) == 1:
            return e
        e += 1
    return None # Should not happen in a valid RSA setup

def generate_private_key(totient, e):
    """
    Generates a private key 'd' such that (d * e) % totient == 1.
    This finds the modular multiplicative inverse of e modulo totient.
    """
    k = 1
    while True:
        # Using the extended Euclidean algorithm property: d = (1 + k * totient) / e
        # We find the first k that results in an integer d.
        if (1 + totient * k) % e == 0:
            d = (1 + totient * k) // e
            return d
        k += 1

def encrypt(message, e, n):
    """Encrypts the message using the public key (e, n)."""
    # Calculates (message ** e) % n
    return pow(message, e, n)

def decrypt(ciphertext, d, n):
    """Decrypts the ciphertext using the private key (d, n)."""
    # Calculates (ciphertext ** d) % n
    return pow(ciphertext, d, n)

# --- Main RSA Execution ---

def main():
    # 1. Generate two distinct prime numbers, p and q
    p = generate_prime()
    q = generate_prime()
    # Ensure p and q are not the same
    while p == q:
        q = generate_prime()

    # 2. Calculate n (modulus) and Euler's totient (phi)
    n = p * q
    totient = (p - 1) * (q - 1)

    # 3. Generate public key 'e'
    e = generate_public_key(totient)

    # 4. Generate private key 'd'
    d = generate_private_key(totient, e)

    # --- Print Key Information ---
    print("--- RSA Key Generation ---")
    print(f"Prime 1 (p): {p}")
    print(f"Prime 2 (q): {q}")
    print(f"Modulus (n = p*q): {n}")
    print(f"Totient (φ = (p-1)*(q-1)): {totient}")
    print(f"Public Key (e): {e}")
    print(f"Private Key (d): {d}")
    print("-" * 28 + "\n")

    # 5. Define the message to be encrypted
    # Note: The message must be smaller than n.
    message = 340

    if message >= n:
        print(f"Error: Message ({message}) is larger than n ({n}). Adjust primes or message.")
        return

    # --- Encryption ---
    print("--- Encryption ---")
    print(f"Original Message: {message}")
    ciphertext = encrypt(message, e, n)
    print(f"Encrypted Ciphertext: {ciphertext}\n")

    # --- Decryption ---
    print("--- Decryption ---")
    decrypted_message = decrypt(ciphertext, d, n)
    print(f"Decrypted Message: {decrypted_message}\n")

    # 7. Verification
    if decrypted_message == message:
        print("✅ Success: The decrypted message matches the original message.")
    else:
        print("❌ Error: Decryption failed.")

# Run the main function when the script is executed
if __name__ == "__main__":
    main()

--- RSA Key Generation ---
Prime 1 (p): 127
Prime 2 (q): 97
Modulus (n = p*q): 12319
Totient (φ = (p-1)*(q-1)): 12096
Public Key (e): 5
Private Key (d): 9677
----------------------------

--- Encryption ---
Original Message: 340
Encrypted Ciphertext: 12122

--- Decryption ---
Decrypted Message: 340

✅ Success: The decrypted message matches the original message.


In [ ]:
!pip install pycryptodome

In [ ]:
import sys
from Crypto.PublicKey import RSA
from Crypto.Random import get_random_bytes
from Crypto.Cipher import AES, PKCS1_OAEP

def main():
    """
    Demonstrates a hybrid encryption scheme using RSA and AES.

    1.  Generate an RSA key pair (public/private).
    2.  Generate a random symmetric session key for AES.
    3.  Encrypt the main message using AES and the session key.
    4.  Encrypt the AES session key using the RSA public key.
    5.  Decrypt the session key using the RSA private key.
    6.  Decrypt the main message using the recovered AES session key.
    """

    try:
        # --- 1. RSA Key Generation ---
        # Generate a 2048-bit RSA key pair.
        # The private key is kept secret by the receiver.
        # The public key is given to the sender.
        print("--- 1. Generating 2048-bit RSA Key Pair ---")
        key = RSA.generate(2048)
        private_key = key
        public_key = key.publickey()

        # print(public_key.export_key().decode()) # Uncomment to see the full public key
        # print(private_key.export_key().decode()) # Uncomment to see the full private key
        print("RSA keys generated successfully.\n")

        # --- 2. Define Message ---
        message = "This is our secret message. It can be very long."
        message_bytes = message.encode("utf-8")
        print(f"--- 2. Original Message ---\n'{message}'\n")

        # --- 3. Encryption (Sender's Side) ---
        print("--- 3. Encrypting Data (Sender's Side) ---")

        # 3a. Generate a random 16-byte (128-bit) session key for AES
        session_key = get_random_bytes(16)

        # 3b. Encrypt the *message* with AES (symmetric encryption)
        # We use AES in EAX mode, which provides confidentiality and authenticity
        cipher_aes = AES.new(session_key, AES.MODE_EAX)
        ciphertext, tag = cipher_aes.encrypt_and_digest(message_bytes)

        # We must store the nonce (number used once) to decrypt later
        # It is not secret and is sent along with the ciphertext
        nonce = cipher_aes.nonce

        # 3c. Encrypt the *session key* with RSA (asymmetric encryption)
        # The sender uses the receiver's public key to encrypt the session key.
        # Only the receiver (with the private key) can decrypt this.
        cipher_rsa = PKCS1_OAEP.new(public_key)
        encrypted_session_key = cipher_rsa.encrypt(session_key)

        print("Encryption complete.\n")

        # --- 4. Transmitted Data ---
        # This is the data that would be sent over a secure channel
        # (e.g., encrypted_session_key, nonce, tag, ciphertext)
        print("--- 4. Data to be Transmitted ---")
        print(f"Encrypted Session Key (RSA):\n{encrypted_session_key.hex()}\n")
        print(f"Nonce (AES):\n{nonce.hex()}\n")
        print(f"Auth Tag (AES):\n{tag.hex()}\n")
        print(f"Ciphertext (AES):\n{ciphertext.hex()}\n")


        # --- 5. Decryption (Receiver's Side) ---
        print("--- 5. Decrypting Data (Receiver's Side) ---")

        # 5a. Decrypt the *session key* with the RSA private key
        # The receiver uses their private key to get the symmetric session key.
        decrypt_rsa = PKCS1_OAEP.new(private_key)
        decrypted_session_key = decrypt_rsa.decrypt(encrypted_session_key)

        # 5b. Decrypt the *message* with the recovered AES session key
        # The receiver uses the recovered session key and the transmitted
        # nonce to create a new AES cipher.
        decrypt_aes = AES.new(decrypted_session_key, AES.MODE_EAX, nonce=nonce)

        # We decrypt and verify the tag simultaneously.
        # If the tag is invalid (data was tampered with), this will raise a ValueError.
        decrypted_message_bytes = decrypt_aes.decrypt_and_verify(ciphertext, tag)
        decrypted_message = decrypted_message_bytes.decode("utf-8")
        print("Decryption and verification successful.\n")

        # --- 6. Verification ---
        print("--- 6. Final Result ---")
        print(f"Decrypted Message:\n'{decrypted_message}'\n")

        # Final check
        if decrypted_message == message:
            print(" Success!! The decrypted message matches the original.")
        else:
            print("Error!! The messages do not match.")

    except (ValueError, TypeError) as e:
        print(f"An error occurred during decryption (e.g., key mismatch or data tampered): {e}", file=sys.stderr)
    except Exception as e:
        print(f"An unexpected error occurred: {e}", file=sys.stderr)

if __name__ == "__main__":
    main()

--- 1. Generating 2048-bit RSA Key Pair ---
RSA keys generated successfully.

--- 2. Original Message ---
'This is our secret message. It can be very long.'

--- 3. Encrypting Data (Sender's Side) ---
Encryption complete.

--- 4. Data to be Transmitted ---
Encrypted Session Key (RSA):
9d41445d308b3539f4aa963afad668d8a8625c43f00f6d882238e841de11b78e394bafefd6d53ca05049ffe5c29f52f35dc4ca2a4d4149037e882c5160d18fa4232d3670d3d8116dd05b3980659517ecac8b3cd00345fb3c6b309142bbb3234db1daa92fbf5a380eb8bace54d1845054d2815b5f306048970ce02b86d1517977475a37a30d6d583cc6f5560412237b9d80c863458a2ee355b2279a96254fe5b74dc1e5b86eec14a215fd5147536f7d07f11e98d84917d9b9ece414ed329ff2333ec264f0888348edd628fd31eba6f3ffc8d056af1137fa72921d9885b431a8b69f42169575e43b3be32307608fd9a534a81e5534120995aa2eda392161afb2e4

Nonce (AES):
ebdc28c6c2884c1b539cce9c540ffb1c

Auth Tag (AES):
58a5bce34be39dd92b5cd7f2a7e0a397

Ciphertext (AES):
c17878e427ec5fdedefc1c7320808fad30f223fd07b6ef8a396639795a71f71db8d42574ed00e5547c422